In [1]:
import sys
sys.path.append('../..')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.metrics import binary_classification as bin_metrics
from src.metrics import multiclass_classification as multi_metrics
from src.models.baselines import MulticlassClassifier

In [3]:
# %env CLEARML_WEB_HOST=https://app.clear.ml
# %env CLEARML_API_HOST=https://api.clear.ml
# %env CLEARML_FILES_HOST=https://files.clear.ml
# %env CLEARML_API_ACCESS_KEY=VT1X7EH5UU1REKI7T44X
# %env CLEARML_API_SECRET_KEY=NHZDqveny2UDyuTvhVADxwUOAgp2mVv3wyipHnKkCc42hiaJog

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=VT1X7EH5UU1REKI7T44X
env: CLEARML_API_SECRET_KEY=NHZDqveny2UDyuTvhVADxwUOAgp2mVv3wyipHnKkCc42hiaJog


In [4]:
# from clearml import Task, Logger

In [5]:
# task = Task.init(
#     project_name='tonality analysis',
#     task_name='TestBaselines',
#     tags=['NaiveBayesClassifier', 'DummyClassifier', 'LogisticRegression'])
# log = Logger.current_logger()

ClearML Task: created new task id=905d9e06a4204c9ab348a8bd71e7194a
2023-04-13 20:47:01,995 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/d279cf59a7ed4fcab0d4f486835b6ad6/experiments/905d9e06a4204c9ab348a8bd71e7194a/output/log


/home/vlad/anaconda3/lib/python3.9/site-packages/nbformat/__init__.py:92: MissingIDFieldWarning:

Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.



In [3]:
path = '../../static/datasets/modified/'
folders = {'bin' : 'bin_classification/', 'multi' : 'multi_classification/'}

In [5]:
#Dummy Classifier
from src.models.baselines import DummyClassifier as Dummy_clf

dummy = Dummy_clf()

train_data = pd.read_csv(path + folders['bin'] + 'train_data.csv')
# task.upload_artifact(name='train_data_for_Dummy', artifact_object='../../../static/datasets/modified/bin_classification/train_data.csv')
# task.upload_artifact(name='val_data_for_Dummy', artifact_object='../../../static/datasets/modified/bin_classification/validation_data.csv')
# task.upload_artifact(name='train_data_for_Dummy', artifact_object='../../../static/datasets/modified/multi_classification/train_data.csv')
# task.upload_artifact(name='val_data_for_Dummy', artifact_object='../../../static/datasets/modified/multi_classification/validation_data.csv')
X_train, y_train = train_data.iloc[:, :-1], train_data.label

dummy.fit(X_train, y_train)

val_data = pd.read_csv(path + folders['bin'] + 'validation_data.csv')
X_val, y_val = val_data.iloc[:, :-1], val_data.label

y_pred = dummy.predict(X_val)

#log.report_scalar("f1-score", "TestBinDummy", value=bin_metrics.f1_score(y_true=y_val, y_pred=y_pred))

print(
    bin_metrics.f1_score(y_true=y_val, y_pred=y_pred)
)


dummy = Dummy_clf()

train_data = pd.read_csv(path + folders['multi'] + 'train_data.csv')
X_train, y_train = train_data.iloc[:, :-1], train_data.label

dummy.fit(X_train, y_train)

val_data = pd.read_csv(path + folders['multi'] + 'validation_data.csv')
X_val, y_val = val_data.iloc[:, :-1], val_data.label

y_pred = dummy.predict(X_val)

#log.report_scalar("f1-score", "TestMultiDummy", value=bin_metrics.f1_score(y_true=y_val, y_pred=y_pred))

print(
    multi_metrics.f1_score(y_true=y_val, y_pred=y_pred)
)

0.4697802197802198
0.5443949262941378


In [6]:
#Naive Bayes
from src.models.baselines import NaiveBayesClassifier as NB

# task.upload_artifact(name='train_data_preprocessed_for_NB', artifact_object='../../../static/datasets/modified/bin_classification/bayes_train.csv')
# task.upload_artifact(name='val_data_preprocessed_for_NB', artifact_object='../../../static/datasets/modified/bin_classification/bayes_val.csv')
# task.upload_artifact(name='train_data_preprocessed_for_NB', artifact_object='../../../static/datasets/modified/multi_classification/bayes_train.csv')
# task.upload_artifact(name='val_data_preprocessed_for_NB', artifact_object='../../../static/datasets/modified/multi_classification/bayes_val.csv')

nb = NB(mode='cat_features')

train_data = pd.read_csv(path + folders['bin'] + 'bayes_train.csv')
X_train, y_train = train_data.iloc[:, :-1], train_data.label

nb.fit(X_train, y_train)

val_data = pd.read_csv(path + folders['bin'] + 'bayes_val.csv')
X_val, y_val = val_data.iloc[:, :-1], val_data.label.values.tolist()

y_pred = nb.predict(X_val)

#log.report_scalar("f1-score", "TestBinNB", value=bin_metrics.f1_score(y_true=y_val, y_pred=y_pred))

print(
    multi_metrics.f1_score(y_true=y_val, y_pred=y_pred),
)

nb = NB(mode='cat_features')

train_data = pd.read_csv(path + folders['multi'] + 'bayes_train.csv')
X_train, y_train = train_data.iloc[:, :-1], train_data.label

nb.fit(X_train, y_train)

val_data = pd.read_csv(path + folders['multi'] + 'bayes_val.csv')
X_val, y_val = val_data.iloc[:, :-1], val_data.label.values.tolist()

X_val, y_val = X_train, y_train.values.tolist()

y_pred = nb.predict(X_val)

#log.report_scalar("f1-score", "TestMultiNB", value=bin_metrics.f1_score(y_true=y_val, y_pred=y_pred))

print(
    multi_metrics.f1_score(y_true=y_val, y_pred=y_pred, average='macro'),
)

800it [00:28, 27.71it/s]


0.67875


6634it [09:08, 12.10it/s]

0.6192820042908561


In [7]:
from sklearn.linear_model import LogisticRegression

# task.upload_artifact(name='train_data_preprocessed_for_LogReg', artifact_object='../../static/embeddings/bin_average_train.csv')
# task.upload_artifact(name='val_data_preprocessed_for_LogReg', artifact_object='../../static/embeddings/bin_average_val.csv')
# task.upload_artifact(name='train_data_preprocessed_for_LogReg', artifact_object='../../static/embeddings/multi_average_train.csv')
# task.upload_artifact(name='val_data_preprocessed_for_LogReg', artifact_object='../../static/embeddings/multi_average_val.csv')

#Binary
logreg = LogisticRegression(C=2, penalty='l2', solver='saga')

train_data = pd.read_csv('../../static/embeddings/bin_average_train.csv')
X_train, y_train = train_data.iloc[:, :-1], train_data.label

logreg.fit(X_train, y_train)

val_data = pd.read_csv('../../static/embeddings/bin_average_val.csv')
X_val, y_val = val_data.iloc[:, :-1], val_data.label

y_pred = logreg.predict(X_val)

#log.report_scalar("f1-score", "TestBinLogReg", value=bin_metrics.f1_score(y_true=y_val, y_pred=y_pred))

print(
    bin_metrics.f1_score(y_true=y_val, y_pred=y_pred),
)

#Multiclass
train_data = pd.read_csv('../../static/embeddings/multi_average_train.csv')
X_train, y_train = train_data.iloc[:, :-1], train_data.label

val_data = pd.read_csv('../../static/embeddings/multi_average_val.csv')
X_val, y_val = val_data.iloc[:, :-1], val_data.label

strategies = ["one-vs-all", "all-vs-all"]

for strategy in strategies:
    multi_clf = MulticlassClassifier(LogisticRegression, strategy)
    multi_clf.fit(X_train, y_train, target_name='label')
    y_pred = multi_clf.predict(X_val)
    #log.report_scalar("f1-score", "TestMultiLogReg strategy:" + strategy, value=bin_metrics.f1_score(y_true=y_val, y_pred=y_pred))
    print(
        multi_metrics.f1_score(y_true=y_val, y_pred=y_pred)
    )

0.7157001414427157
0.6927877947295423
0.7060439560439561


In [10]:
# task.close()

/home/vlad/anaconda3/lib/python3.9/site-packages/nbformat/__init__.py:92: MissingIDFieldWarning:

Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.

